In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

## Organize imports

In [3]:
from pathlib import Path
from datetime import datetime

In [4]:
import numpy as np

In [5]:
from collections import OrderedDict

In [6]:
import multiprocessing

In [7]:
import PIL
from PIL.Image import Image

In [8]:
import matplotlib.pyplot as plt

In [9]:
import torch
import torch.nn.functional as F
from torch import nn, Tensor, optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
import torchvision
from torchvision import (transforms, datasets)
from torchvision import models 

```python
! conda install pytorch torchvision torchaudio -c pytorch
```

```python
! pip install pytorch-lightning
```

In [10]:
from pytorch_lightning import LightningModule, Trainer

In [11]:
from torchmetrics import Accuracy

## Configure environment

In [20]:
workers = multiprocessing.cpu_count()
workers

8

In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [22]:
path = Path('data')
path.mkdir(exist_ok=True)

## Train MNIST classifier

In [23]:
epochs = 16
lr = 0.1
bs = 2 * 64
gamma = 0.7
save_model = False
log_interval = 10
dry_run = False

In [24]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        
        return output

In [25]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            if dry_run:
                break

In [26]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [29]:
use_cuda = torch.cuda.is_available()

torch.manual_seed(2022)

device = torch.device("cuda" if use_cuda else "cpu")

train_kwargs = {'batch_size': 2 * 64}
test_kwargs = {'batch_size': 1000}
if use_cuda:
    cuda_kwargs = {'num_workers': 1,
                   'pin_memory': True,
                   'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])
dataset1 = datasets.MNIST(path / 'mnist', train=True, download=True,
                   transform=transform)
dataset2 = datasets.MNIST(path / 'mnist', train=False,
                   transform=transform)
train_loader = DataLoader(dataset1, **train_kwargs)
test_loader = DataLoader(dataset2, **test_kwargs)

model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=lr)

scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=gamma)

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

if save_model:
    torch.save(model.state_dict(), path / 'mnist' / 'mnist_cnn.pt')

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.298131
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.825917
Train Epoch: 1 [1280/60000 (2%)]	Loss: 1.210144
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.943850
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.885442
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.628774
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.852532
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.514832
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.663228
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.546291
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.369553
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.382517
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.345922
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.581012
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.551198
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.285307
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.587430
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.492382
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.152887
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.157820
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.081467
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.026983
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.029528
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.174082
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.122709
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.070154
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.083151
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.044301
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.055816
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.208419
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.032888
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.179407
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.199686
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.023887
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.037393
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.094247
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.030548
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.035678
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.133128
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.035359


Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.146458
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.085150
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.012985
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.057992
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.013348
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.072572
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.053648
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.111272
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.045711
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.056876
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.030068
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.020170
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.075423
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.030893
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.090687
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.050275
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.052832
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.047415
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.121579
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.081532


Train Epoch: 6 [12800/60000 (21%)]	Loss: 0.126920
Train Epoch: 6 [13440/60000 (22%)]	Loss: 0.018149
Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.043758
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.094003
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.026484
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.036120
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.149191
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.047744
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.035456
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.013141
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.041890
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.017299
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.058239
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.041384
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.020726
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.080272
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.017536
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.021299
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.084909
Train Epoch: 6 [24960/60000 (42%)]	Loss: 0.012700


Train Epoch: 7 [57600/60000 (96%)]	Loss: 0.057735
Train Epoch: 7 [58240/60000 (97%)]	Loss: 0.116418
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.040386
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.140920

Test set: Average loss: 0.0380, Accuracy: 9872/10000 (99%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.012101
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.035058
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.029777
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.020061
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.124362
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.021006
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.014084
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.021637
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.086772
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.174961
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.215024
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.019618
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.104898
Train Epoch: 8 [8320/60000 (14%)]	Loss: 0.036012
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.011140
Train Epoch: 8 [9

Train Epoch: 9 [41600/60000 (69%)]	Loss: 0.063448
Train Epoch: 9 [42240/60000 (70%)]	Loss: 0.010434
Train Epoch: 9 [42880/60000 (71%)]	Loss: 0.008941
Train Epoch: 9 [43520/60000 (72%)]	Loss: 0.204202
Train Epoch: 9 [44160/60000 (74%)]	Loss: 0.066305
Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.016946
Train Epoch: 9 [45440/60000 (76%)]	Loss: 0.073747
Train Epoch: 9 [46080/60000 (77%)]	Loss: 0.015461
Train Epoch: 9 [46720/60000 (78%)]	Loss: 0.088175
Train Epoch: 9 [47360/60000 (79%)]	Loss: 0.004959
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.050445
Train Epoch: 9 [48640/60000 (81%)]	Loss: 0.044972
Train Epoch: 9 [49280/60000 (82%)]	Loss: 0.031738
Train Epoch: 9 [49920/60000 (83%)]	Loss: 0.157733
Train Epoch: 9 [50560/60000 (84%)]	Loss: 0.018779
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.026275
Train Epoch: 9 [51840/60000 (86%)]	Loss: 0.005043
Train Epoch: 9 [52480/60000 (87%)]	Loss: 0.071828
Train Epoch: 9 [53120/60000 (88%)]	Loss: 0.060817
Train Epoch: 9 [53760/60000 (90%)]	Loss: 0.066188


Train Epoch: 11 [23680/60000 (39%)]	Loss: 0.027014
Train Epoch: 11 [24320/60000 (41%)]	Loss: 0.118115
Train Epoch: 11 [24960/60000 (42%)]	Loss: 0.141170
Train Epoch: 11 [25600/60000 (43%)]	Loss: 0.029257
Train Epoch: 11 [26240/60000 (44%)]	Loss: 0.023919
Train Epoch: 11 [26880/60000 (45%)]	Loss: 0.267834
Train Epoch: 11 [27520/60000 (46%)]	Loss: 0.007144
Train Epoch: 11 [28160/60000 (47%)]	Loss: 0.130762
Train Epoch: 11 [28800/60000 (48%)]	Loss: 0.076435
Train Epoch: 11 [29440/60000 (49%)]	Loss: 0.017946
Train Epoch: 11 [30080/60000 (50%)]	Loss: 0.039395
Train Epoch: 11 [30720/60000 (51%)]	Loss: 0.018130
Train Epoch: 11 [31360/60000 (52%)]	Loss: 0.095822
Train Epoch: 11 [32000/60000 (53%)]	Loss: 0.072353
Train Epoch: 11 [32640/60000 (54%)]	Loss: 0.019555
Train Epoch: 11 [33280/60000 (55%)]	Loss: 0.087774
Train Epoch: 11 [33920/60000 (57%)]	Loss: 0.035154
Train Epoch: 11 [34560/60000 (58%)]	Loss: 0.026557
Train Epoch: 11 [35200/60000 (59%)]	Loss: 0.060714
Train Epoch: 11 [35840/60000 (6

Train Epoch: 13 [5760/60000 (10%)]	Loss: 0.017623
Train Epoch: 13 [6400/60000 (11%)]	Loss: 0.067351
Train Epoch: 13 [7040/60000 (12%)]	Loss: 0.031262
Train Epoch: 13 [7680/60000 (13%)]	Loss: 0.016828
Train Epoch: 13 [8320/60000 (14%)]	Loss: 0.107384
Train Epoch: 13 [8960/60000 (15%)]	Loss: 0.073274
Train Epoch: 13 [9600/60000 (16%)]	Loss: 0.072028
Train Epoch: 13 [10240/60000 (17%)]	Loss: 0.017689
Train Epoch: 13 [10880/60000 (18%)]	Loss: 0.049223
Train Epoch: 13 [11520/60000 (19%)]	Loss: 0.060404
Train Epoch: 13 [12160/60000 (20%)]	Loss: 0.025877
Train Epoch: 13 [12800/60000 (21%)]	Loss: 0.007500
Train Epoch: 13 [13440/60000 (22%)]	Loss: 0.053719
Train Epoch: 13 [14080/60000 (23%)]	Loss: 0.014229
Train Epoch: 13 [14720/60000 (25%)]	Loss: 0.075385
Train Epoch: 13 [15360/60000 (26%)]	Loss: 0.071272
Train Epoch: 13 [16000/60000 (27%)]	Loss: 0.078588
Train Epoch: 13 [16640/60000 (28%)]	Loss: 0.057084
Train Epoch: 13 [17280/60000 (29%)]	Loss: 0.167235
Train Epoch: 13 [17920/60000 (30%)]	Lo

Train Epoch: 14 [48640/60000 (81%)]	Loss: 0.023392
Train Epoch: 14 [49280/60000 (82%)]	Loss: 0.066030
Train Epoch: 14 [49920/60000 (83%)]	Loss: 0.046026
Train Epoch: 14 [50560/60000 (84%)]	Loss: 0.031265
Train Epoch: 14 [51200/60000 (85%)]	Loss: 0.066087
Train Epoch: 14 [51840/60000 (86%)]	Loss: 0.041565
Train Epoch: 14 [52480/60000 (87%)]	Loss: 0.059493
Train Epoch: 14 [53120/60000 (88%)]	Loss: 0.044823
Train Epoch: 14 [53760/60000 (90%)]	Loss: 0.084210
Train Epoch: 14 [54400/60000 (91%)]	Loss: 0.076470
Train Epoch: 14 [55040/60000 (92%)]	Loss: 0.056437
Train Epoch: 14 [55680/60000 (93%)]	Loss: 0.131999
Train Epoch: 14 [56320/60000 (94%)]	Loss: 0.013175
Train Epoch: 14 [56960/60000 (95%)]	Loss: 0.211623
Train Epoch: 14 [57600/60000 (96%)]	Loss: 0.034819
Train Epoch: 14 [58240/60000 (97%)]	Loss: 0.178766
Train Epoch: 14 [58880/60000 (98%)]	Loss: 0.008415
Train Epoch: 14 [59520/60000 (99%)]	Loss: 0.022785

Test set: Average loss: 0.0369, Accuracy: 9875/10000 (99%)

Train Epoch: 15 [0/60

Train Epoch: 16 [30720/60000 (51%)]	Loss: 0.042133
Train Epoch: 16 [31360/60000 (52%)]	Loss: 0.045969
Train Epoch: 16 [32000/60000 (53%)]	Loss: 0.154172
Train Epoch: 16 [32640/60000 (54%)]	Loss: 0.221706
Train Epoch: 16 [33280/60000 (55%)]	Loss: 0.163963
Train Epoch: 16 [33920/60000 (57%)]	Loss: 0.044652
Train Epoch: 16 [34560/60000 (58%)]	Loss: 0.035362
Train Epoch: 16 [35200/60000 (59%)]	Loss: 0.011358
Train Epoch: 16 [35840/60000 (60%)]	Loss: 0.085615
Train Epoch: 16 [36480/60000 (61%)]	Loss: 0.105816
Train Epoch: 16 [37120/60000 (62%)]	Loss: 0.074874
Train Epoch: 16 [37760/60000 (63%)]	Loss: 0.050208
Train Epoch: 16 [38400/60000 (64%)]	Loss: 0.088319
Train Epoch: 16 [39040/60000 (65%)]	Loss: 0.011977
Train Epoch: 16 [39680/60000 (66%)]	Loss: 0.219594
Train Epoch: 16 [40320/60000 (67%)]	Loss: 0.050883
Train Epoch: 16 [40960/60000 (68%)]	Loss: 0.093366
Train Epoch: 16 [41600/60000 (69%)]	Loss: 0.041924
Train Epoch: 16 [42240/60000 (70%)]	Loss: 0.104504
Train Epoch: 16 [42880/60000 (7

## Train letters classificartion with convolutional neural networkm

#### Prepare data

In [ ]:
tfms = transforms.Compose([transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))])

In [ ]:
image_path = path / 'geomnist_dataset'

In [ ]:
geoletters = path / 'geoletters'
geoletters.mkdir(exist_ok=True)

In [ ]:
# ! cp /content/drive/My\ Drive/datasets/letters/trained_data/geomnist_dataset.zip {path}

In [ ]:
# ! wget https://github.com/MaxinAI/school-of-ai/raw/master/data/geoletters/geomnist_dataset.zip {geoletters}

In [ ]:
# import zipfile
# with zipfile.ZipFile(path / 'geoletters' / 'geomnist_dataset.zip' , 'r') as zip_ref:
#     zip_ref.extractall(path)

## Prepare data loaders

In [ ]:
def img_loader(img_path:Path):
    with open(img_path, mode='rb') as fl:
        with PIL.Image.open(fl) as img:
            return img.convert('L')

In [ ]:
train_dataset = datasets.ImageFolder(image_path / 'train_geo', loader=img_loader, transform=tfms)
valid_dataset = datasets.ImageFolder(image_path / 'val_geo', loader=img_loader, transform=tfms)
test_dataset = datasets.ImageFolder(image_path / 'test_geo', loader=img_loader, transform=tfms)

In [ ]:
train_dataset, valid_dataset, train_dataset

In [ ]:
# ?DataLoader

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, 
                          num_workers=workers, drop_last=True)
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False, 
                          num_workers=workers)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False,
                        num_workers=workers)

In [ ]:
test_dataset[0][0].shape

In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.1307])
    std = np.array([0.3081])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.figure(figsize=(64, 64))
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(iter(train_loader))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[classes[x] for x in classes])

In [ ]:
loss_func = nn.CrossEntropyLoss()

#### Model initialization

In [ ]:
# ?? nn.Flatten

In [ ]:
input_channels = 1

In [ ]:
lower_body = nn.Sequential(OrderedDict([('conv1', nn.Conv2d(input_channels, 32, 3)),
                                        ('bn1', nn.BatchNorm2d(32)),
                                        ('relu1', nn.ReLU(inplace=True)),
                                        ('mxpl1', nn.MaxPool2d(2, 2)),
                                        ('conv2', nn.Conv2d(32, 64, kernel_size=3)),
                                        ('bn2', nn.BatchNorm2d(64)),
                                        ('relu2', nn.ReLU(inplace=True)),
                                        ('mxpl2', nn.MaxPool2d(2, 2)),
                                        ('drop1', nn.Dropout2d(p=0.25))]))

In [ ]:
conv_body = nn.Sequential(OrderedDict([('conv3', nn.Conv2d(64, 128, kernel_size=3)),
                                       ('bn3', nn.BatchNorm2d(128)),
                                       ('relu3', nn.ReLU(inplace=True)),
                                       ('mxpl3', nn.MaxPool2d(2, 2)),
                                       ('drop2', nn.Dropout2d(p=0.25))]))

In [ ]:
linear_body = nn.Sequential(OrderedDict([('flatten', nn.Flatten()),
                                         ('ln1', nn.Linear(2 * 2 * 128, 1024, bias=True)),
                                         ('bn2', nn.BatchNorm1d(1024)),
                                         ('relu3', nn.ReLU(inplace=True)),
                                         ('drop2', nn.Dropout(p=0.25))]))

In [ ]:
class LetterNet(nn.Module):
    """Full double letters network implementation"""

    def __init__(self, input_channels=1, num_classes=33):
        super(LetterNet, self).__init__()
        self.conv_part = lower_body
        self.dub_part = conv_body
        self.fc_part = linear_body
        self.fc = nn.Linear(1024, num_classes)

    def forward(self, x):
        x = self.conv_part(x)
        x = self.dub_part(x)
        x = self.fc_part(x)
        logits = self.fc(x)

        return logits



In [ ]:
def conv_out(w:int, k: int, p: int, s: int):
    return int(((w - k + 2*p)/s)+1)

In [ ]:
conv_out(32, 3, 0, 1)

In [ ]:
conv_out(conv_out(conv_out(conv_out(32, 3, 0, 1), 2, 0, 1), 3, 0, 1), 2, 0, 1)

In [ ]:
class LetterNet2(nn.Module):
    """Full double letters network implementation"""

    def __init__(self, input_channels=1, num_classes=33):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, 32, 3)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc = nn.Linear(6 * 6 * 64, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = F.relu(x)
#         logits = self.pool2(x)
        x = self.pool2(x)
        x = torch.flatten(x, start_dim=1)
        logits = self.fc(x)

        return logits

In [ ]:
sz = 32 # 34
x_test = torch.randn(4 , 1, sz, sz)

In [ ]:
model = LetterNet2()

In [ ]:
o_test = model(x_test)
o_test.shape

In [ ]:
model = LetterNet()

In [ ]:
model

## Create lightning module

In [ ]:
class ClassifierNetLt(LightningModule):
    def __init__(
        self, model: nn.Module, loss_fn=loss_func, metrics=Accuracy()):
        super().__init__()
        self.model = model
        self.accuracy = metrics
        self.lr=0.01
        self.loss_fn = loss_fn

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_nb):
        x, y = batch
        z = self(x)
        loss = self.loss_fn(z, y)
        
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        z = self(x)
        loss = self.loss_fn(z, y)
        preds = torch.argmax(z, dim=1)
        self.accuracy(preds, y)

        # Calling self.log will surface up scalars for you in TensorBoard
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", self.accuracy, prog_bar=True)
        
        return loss
    
    def test_step(self, batch, batch_idx):
        # Here we just reuse the validation_step for testing
        return self.validation_step(batch, batch_idx)

    def configure_optimizers(self):
        optimizer = optim.Adam(self.model.parameters(), lr=self.lr)
        lr_scheduler_cos = lr_scheduler.CosineAnnealingWarmRestarts(
            optimizer, 
            T_0=10, T_mult=2, 
            eta_min=0.0001, 
            last_epoch=-1,
            verbose=True)
        
        return [optimizer], [lr_scheduler_cos]


In [ ]:
AVAIL_GPUS = min(1, torch.cuda.device_count())
AVAIL_GPUS

In [ ]:
model_lt = ClassifierNetLt(model)
trainer = Trainer(
    gpus=AVAIL_GPUS,
    max_epochs=8,
    progress_bar_refresh_rate=20,
    auto_lr_find=True,
)

In [ ]:
! ls {geoletters}

In [ ]:
curr_date = datetime.now().strftime('%d_%m_%Y').lower()
curr_date

In [ ]:
geletters_ckpts = geoletters / f'geoletters_{curr_date}'
geletters_ckpts.mkdir(exist_ok=True)

## Train model

In [ ]:
trainer.fit(
    model_lt, train_loader, valid_loader)

#### Find learning rate

In [ ]:
model_lt_lr = ClassifierNetLt(LetterNet())
trainer_lr = Trainer(
    gpus=AVAIL_GPUS,
    max_epochs=3,
    progress_bar_refresh_rate=20,
    auto_lr_find=True,
)

In [ ]:
lr_finder = trainer_lr.tuner.lr_find(
    model_lt_lr, train_loader, valid_loader)

In [ ]:
# Results can be found in
lr_finder.results

In [ ]:
# Plot with
fig = lr_finder.plot(suggest=True)
fig.show()

In [ ]:
# Pick point based on plot, or get suggestion
new_lr = lr_finder.suggestion()

# update hparams of the model
model_lt_lr.lr = new_lr
new_lr

In [ ]:
trainer_lr.tune(
    model_lt_lr, train_loader, valid_loader)

#### Size dependency

In [ ]:
model

In [ ]:
sz = 32 # 34
x_test = torch.randn(4 , 1, sz, sz)

In [ ]:
model_lt.freeze()
model_lt(x_test)

## Train Bigger Model

In [ ]:
resnet = models.resnet18(pretrained=False, progress=True)
resnet

In [ ]:
resnet.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
resnet

#### Re-initialize data loaders

In [ ]:
bs = 2 * 64

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=bs, shuffle=True, 
                          num_workers=workers, drop_last=True)
valid_loader = DataLoader(valid_dataset, batch_size=bs, shuffle=False, 
                          num_workers=workers)
test_loader = DataLoader(test_dataset, batch_size=bs, shuffle=False,
                        num_workers=workers)

In [ ]:
resnet_lt = ClassifierNetLt(resnet)
trainer = Trainer(
    gpus=AVAIL_GPUS,
    benchmark=True,
    max_epochs=16,
    progress_bar_refresh_rate=20,
    auto_lr_find=True,
    stochastic_weight_avg=True,
)

In [ ]:
trainer.fit(
    resnet_lt, train_loader, valid_loader)

#### Save the model state dict

In [ ]:
net = resnet_lt.model

In [ ]:
net.state_dict()

In [ ]:
torch.save(resnet_lt.model.state_dict(), str(geletters_ckpts / 'resnet_18_state_dict.pth'))

In [ ]:
resnet_ld = models.resnet18(pretrained=False, progress=False)
resnet_ld.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

In [ ]:
state_dict = torch.load(str(geletters_ckpts / 'resnet_18_state_dict.pth'), map_location='cpu')

In [ ]:
resnet_ld.load_state_dict(state_dict)

In [ ]:
geletters_ckpts

In [ ]:
! ls {geletters_ckpts}

#### Save entire model

In [ ]:
torch.save(resnet_lt.model, geletters_ckpts / 'resnet_18_entire.pth')

In [ ]:
resnet_ld = torch.load(geletters_ckpts / 'resnet_18_entire.pth')

In [ ]:
# resnet_ld.state_dict(), resnet_lt.model.state_dict()

In [ ]:
resnet_ld

## Let's try to inference

Run inference service in different session

! pip install -U flask

In [ ]:
geletters_ckpts / 'resnet_18_entire.pth'

In [ ]:
! ls data/geoletters/geoletters_labels.json

In [ ]:
geletters_ckpts.absolute()

## Image search

In [ ]:
from typing import Union

In [ ]:
from tqdm import tqdm

In [ ]:
import cv2

In [ ]:
import PIL
from PIL import Image

In [ ]:
from torch import no_grad
from torch.jit import ScriptModule
from torchvision.models import (resnet34, resnet50, wide_resnet50_2)

In [ ]:
size = 256
imsz = 224
IMG_SUFF = {'.jpg', '.jpeg', '.png'}

In [ ]:
class ToPILImage(object):
    """Convert inout image to PIL image"""

    def __init__(self, mode=None):
        super().__init__()
        self.to_pil = transforms.ToPILImage(mode=mode)

    def convert(self, img: Union[np.ndarray, PIL.Image.Image]):
        """
        Converts image to the PIL format
        Args:
            img: inout image

        Returns:
            converted image
        """
        return img if isinstance(img, PIL.Image.Image) else self.to_pil(img)

    def __call__(self, *args, **kwargs):
        return self.convert(*args, **kwargs)

    def __repr__(self):
        format_string = self.__class__.__name__ + '('
        if self.to_pil.mode is not None:
            format_string += f'mode={self.to_pil.mode}'
        format_string += ')'
        return format_string


class Img2Vec(object):
    """Model wrapper for image embedding"""

    def __init__(self, backbone: Union[nn.Module, ScriptModule], trfm: transforms, device: str = 'cpu'):
        super().__init__()
        self.device = torch.device(device)
        self.backbone = (backbone.eval() if hasattr(backbone, 'eval') else backbone).to(device)
        self.trfm = trfm

    def preprocess(self, *xs: Union[np.ndarray, PIL.Image.Image]) -> Tensor:
        """
        Transform data before model
        Args:
            *xs: input data

        Returns:
            processed data for model
        """
        return torch.stack([self.trfm(x) for x in xs]).to(self.device)

    @no_grad()
    def forward(self, *xs: np.ndarray) -> np.ndarray:
        tns = self.preprocess(*xs)
        rts = self.backbone(tns)
        y = rts.cpu().data.numpy()

        return y

    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)

In [ ]:
vec_trsfm = transforms.Compose([ToPILImage(mode='RGB'),
                                transforms.Resize(size),
                                transforms.CenterCrop(imsz),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

#### Prepare data

In [ ]:
search_path = path / 'search'

In [ ]:
dir_paths = [dp for dp in search_path.iterdir() if dp.is_dir()]

In [ ]:
dir_paths

In [ ]:
img_pts = [im_pt for dp in dir_paths for im_pt in dp.iterdir() if im_pt.suffix in IMG_SUFF]

In [ ]:
def read_img(im_pt):
    img = cv2.imread(str(im_pt), cv2.IMREAD_ANYCOLOR)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    
    return img

In [ ]:
imgs = [read_img(ip) for ip in img_pts]

#### Initialize features extractor

In [ ]:
cut = 1

In [ ]:
body = wide_resnet50_2(pretrained=True)

In [ ]:
body

In [ ]:
backbone = nn.Sequential(*list(body.children())[:-cut])
net = nn.Sequential(backbone, nn.Flatten())

In [ ]:
net

In [ ]:
img_vec = Img2Vec(net, vec_trsfm, device='cpu')

In [ ]:
with tqdm(imgs, desc='Vectorizing images') as p_imgs:
    vecs = [img_vec(im)[0] for im in p_imgs]

In [ ]:
vecs[0].shape, len(vecs)

In [ ]:
img_vecs = list(zip(imgs, vecs))

In [ ]:
img_vecs

#### Compare vectors

! pip install scipy

In [ ]:
from scipy.spatial.distance import cosine

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def top_vecs(qi, vec_func=img_vec, db=img_vecs, top_k=5):
    qv = vec_func(qi)[0]
    resul_pts = [(cosine(qv, vc), pt) for pt, vc in db]
    resul_pts = sorted(resul_pts, key=lambda x: x[0], reverse=False)
    resul_pts = resul_pts[:top_k]
    
    return resul_pts

#### Query images

In [ ]:
query_path = path / 'queries'
query_path.mkdir(exist_ok=True)

In [ ]:
qim = read_img(query_path / 'ct_2.jpg')

In [ ]:
img_vec(imgs[0])

In [ ]:
res = top_vecs(qim, vec_func=img_vec, db=img_vecs)

In [ ]:
plt.imshow(qim)
plt.show()
plt.close()

In [ ]:
for dist, res_img in res:
    plt.imshow(res_img)
    plt.show()
    plt.close()

#### Image similarity with CLIP model

! pip install git+https://github.com/openai/CLIP.git

In [ ]:
import clip

In [ ]:
model, preprocess = clip.load("ViT-B/32", device=device)

In [ ]:
model.eval()

In [ ]:
img_vec_clip = Img2Vec(model, preprocess, device=device)

In [ ]:
img_vec_clip.backbone = img_vec_clip.backbone.encode_image

In [ ]:
with tqdm(imgs, desc='Reading images') as p_paths:
    pil_imgs = [Image.fromarray(im) for im in imgs]

In [ ]:
pil_imgs

In [ ]:
# ??preprocess.transforms

In [ ]:
# preprocess(pil_imgs[0])

In [ ]:
with tqdm(pil_imgs, desc='Vectorizing images') as p_imgs:
    clip_vecs = [img_vec_clip(im)[0] for im in p_imgs]

In [ ]:
clip_img_vecs = list(zip(imgs, clip_vecs))

In [ ]:
clip_vecs[0].shape

#### Query images

In [ ]:
query_path = path / 'queries'
query_path.mkdir(exist_ok=True)

In [ ]:
qim = read_img(query_path / 'ct_2.jpg')

In [ ]:
res = top_vecs(Image.fromarray(qim), vec_func=img_vec_clip, db=clip_img_vecs)

In [ ]:
#res

In [ ]:
plt.imshow(qim)
plt.show()
plt.close()

In [ ]:
for dist, res_img in res:
    plt.imshow(res_img)
    plt.show()
    plt.close()

## Fine-tuning model

#### Dlownload and store data

In [ ]:
hymenoptera = path / 'hymenoptera'
hymenoptera.mkdir(exist_ok=True)
hymenoptera

In [ ]:
! wget https://download.pytorch.org/tutorial/hymenoptera_data.zip -P {hymenoptera}

In [ ]:
! ls {hymenoptera}

In [ ]:
import zipfile
with zipfile.ZipFile(hymenoptera / 'hymenoptera_data.zip' , 'r') as zip_ref:
    zip_ref.extractall(path)

In [ ]:
! ls {path / 'hymenoptera_data'}

#### Initialize data loaders

In [ ]:
# Data augmentation and normalization for training
# Just normalization for validation
train_transforms = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
val_transforms = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

data_dir = path / 'hymenoptera_data'

train_dataset = datasets.ImageFolder(data_dir / 'train', train_transforms)
val_dataset = datasets.ImageFolder(data_dir / 'val', val_transforms)

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, 
                          num_workers=workers, drop_last=True)
val_datloader = DataLoader(val_dataset, batch_size=64, shuffle=False, 
                          num_workers=workers)
test_datloader = DataLoader(val_dataset, batch_size=4, shuffle=True, 
                          num_workers=workers)

dataset_sizes = len(train_dataloader) + len(val_datloader)
class_names = train_dataset.classes

In [ ]:
class_names

In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.figure(figsize=(32, 32))
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(iter(train_dataloader))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])

In [ ]:
def visualize_model(model, num_images=6):
    model.eval()
    images_so_far = 0
    fig = plt.figure()
    model.freeze()
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(test_datloader):
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title(f'predicted: {class_names[preds[j]]} ground true: {class_names[labels[j]]}')
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    return


In [ ]:
for i, (inputs, labels) in enumerate(test_datloader):
    print(i, labels)

#### Initialize the model for fine-tuning

In [ ]:
model_ft = models.resnet18(pretrained=True, progress=True)
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, 2)

In [ ]:
class ClassifierHymLt(ClassifierNetLt):
    
    def configure_optimizers(self):
        optimizer_ft = optim.SGD(
            self.model.parameters(), lr=0.001, momentum=0.9)
        exp_lr_scheduler = lr_scheduler.StepLR(
            optimizer_ft, step_size=7, gamma=0.1)
        
        return [optimizer_ft], [exp_lr_scheduler]

In [ ]:
model_ft_lt = ClassifierHymLt(model_ft)
trainer = Trainer(
    gpus=AVAIL_GPUS,
    benchmark=True,
    max_epochs=32,
    progress_bar_refresh_rate=20,
    auto_lr_find=True,
#     stochastic_weight_avg=True,
)

In [ ]:
trainer.fit(model_ft_lt, train_dataloader, val_datloader)

In [ ]:
model_ft_lt

In [ ]:
visualize_model(model_ft_lt, num_images=12)

#### Transfer learning

In [ ]:
model_conv = models.resnet18(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 2)

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

In [ ]:
class ClassifierHymFtLt(ClassifierNetLt):
    
    def configure_optimizers(self):
        optimizer_ft = optim.SGD(
            self.model.fc.parameters(), lr=0.001, momentum=0.9)
        exp_lr_scheduler = lr_scheduler.StepLR(
            optimizer_ft, step_size=7, gamma=0.1)
        
        return [optimizer_ft], [exp_lr_scheduler]

In [ ]:
model_ft_lt = ClassifierHymFtLt(model_conv)
trainer = Trainer(
    gpus=AVAIL_GPUS,
    benchmark=True,
    max_epochs=32,
    progress_bar_refresh_rate=20,
    auto_lr_find=True,
#     stochastic_weight_avg=True,
)

In [ ]:
trainer.fit(model_ft_lt, train_dataloader, val_datloader)

In [ ]:
visualize_model(model_ft_lt, num_images=12)